In [1]:
import xarray as xr
import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colorbar, colors
from matplotlib.cm import get_cmap
from matplotlib.patches import Rectangle
import seawater
import glob

In [2]:
%%time
files = glob.glob("/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/*.nc")

CPU times: user 585 µs, sys: 654 µs, total: 1.24 ms
Wall time: 1.12 ms


In [ ]:
%%time
# files = glob.glob("/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/*.nc")
# variable_1 = 'Alk'
# variable_2 =  'DIC'
# datasets = []
# for f in files:
#     a = xr.open_dataset(f)
#     a_new = a[[variable_1, variable_2]]
#     datasets.append(a_new)
# x = xr.concat(datasets,dim='time')


variable_1 = 'Alk'
variable_2 =  'DIC'

file_list=os.listdir(r"/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/")

def preprocess(ds):
    return ds[[variable_1,variable_2]]

x = xr.open_mfdataset('/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/'+file_list[0], combine='nested', concat_dim=["time"], preprocess=preprocess, chunks={})
dslice = x.where(x['Alk'] != 0)
dslice = dslice.sel(eta_rho=slice(502,522), xi_rho=slice(595,610)).mean(dim=('time', 'xi_rho', 'eta_rho'))
dslice = dslice.load()

for i in range (1,len(file_list)):
        x = xr.open_mfdataset('/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/'+file_list[i], combine='nested', concat_dim=["time"], preprocess=preprocess, chunks={})
        dslice1 = x.where(x['Alk'] != 0)
        dslice1 = dslice1.sel(eta_rho=slice(502,522), xi_rho=slice(595,610)).mean(dim=('time', 'xi_rho', 'eta_rho'))
        dslice1 = dslice1.load()
        dslice = xr.concat([dslice,dslice1],dim='time')
    
    

In [ ]:
# ds = xr.open_mfdataset("/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/*.nc", combine='nested', concat_dim=["time"])
# # x=xr.open_mfdataset('/expanse/lustre/projects/cos100/uheede/roms/run/Iceland4_BGC_alk/Iceland4_BGC_alk_rnd.2012062[2-3]150000.nc', combine='nested', concat_dim=["time"])
# x = ds[['Alk','DIC']]

In [ ]:
# %%time
# variable_1 = 'Alk'
# variable_2 =  'DIC'
# def preprocess(ds):
#     return ds[[variable_1,variable_2]]
# x = xr.open_mfdataset("/expanse/lustre/projects/cos100/slevy1/icelandBGCdata/*.nc", combine='nested', concat_dim=["time"], preprocess=preprocess, chunks={})


In [ ]:
print(x.data_vars)

In [ ]:
%%time
dslice = x.where(x['Alk'] != 0)
dslice = dslice.sel(eta_rho=slice(502,522), xi_rho=slice(595,610)).mean(dim=('time', 'xi_rho', 'eta_rho'))

In [ ]:
%%time
alkalinity = dslice['Alk']
depth = dslice['depth']
DIC = dslice['DIC']

# Create a plot of alkalinity vs. depth
plt.figure(figsize=(6, 8))
plt.plot(alkalinity, depth)


plt.xlabel('Alkalinity')
plt.ylabel('Depth')
plt.title('Alkalinity as a Function of Depth')
plt.show

plt.figure(figsize=(6, 8))
plt.plot(DIC, depth)

plt.xlabel('DIC')
plt.ylabel('Depth')
plt.title('DIC as a Function of Depth')
plt.show

In [ ]:
%%time
top = x.sel(depth=0).mean(dim='time')
top = top.where(top['Alk'] != 0)

In [ ]:
%%time
alk_surface = top['Alk']
dic_surface = top['DIC']

rectangle_alk = Rectangle((595, 502), 15, 20, linewidth=1, edgecolor='r', facecolor='none')

# Create the plot
plt.figure(figsize=(10, 10))
alk_surface.plot(x='xi_rho', y='eta_rho')
plt.title('Surface Alkalinity (Alk) Distribution')
plt.xlabel('xi_rho')
plt.ylabel('eta_rho')
plt.gca().add_patch(rectangle_alk)
plt.show()

rectangle_dic = Rectangle((595, 502), 15, 20, linewidth=1, edgecolor='r', facecolor='none')

# Create the plot
plt.figure(figsize=(10, 10))
dic_surface.plot(x='xi_rho', y='eta_rho')
plt.title('Surface DIC Distribution')
plt.xlabel('xi_rho')
plt.ylabel('eta_rho')
plt.gca().add_patch(rectangle_dic)
plt.show()

alk_flat = alk_surface.values.flatten()
alk_flat_clean = alk_flat[~np.isnan(alk_flat)]
dic_flat = dic_surface.values.flatten()
dic_flat_clean = dic_flat[~np.isnan(dic_flat)]

data_to_plot = [alk_flat_clean, dic_flat_clean]

plt.figure(figsize=(10, 6))
plt.boxplot(data_to_plot, labels=['Alk', 'DIC'])
plt.title('Box Plot of Surface Alkalinity (Alk) and Dissolved Inorganic Carbon (DIC)')
plt.ylabel('Value')
plt.show()